<a href="https://colab.research.google.com/github/brendan123/Semantic-Segmentation/blob/milestone-1/SemanticSegementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone 1: Environment Preparation

### Install Dependencies

In [3]:
# Python dependencies
! pip install nni # install nni
! pip install patchify
! pip install segmentation-models

# Ngrok
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip # download ngrok and unzip it
! unzip ngrok-stable-linux-amd64.zip

# NNI
! mkdir -p nni_repo
! git clone https://github.com/microsoft/nni.git nni_repo/nni # clone NNI's offical repo to get examples

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-10-25 23:08:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  5.97MB/s    in 2.2s    

2022-10-25 23:08:23 (5.97 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
fatal: destination path 'n

### NNI UI

In [ ]:
# Kills old ngrok instance running on 5000
! kill $(lsof -t -i:5000)
# This is my authtoken please don't take it
! ./ngrok authtoken 26JQrkyWKQCjAu3oESdi9hlbctZ_4VPqxWtWrM7YaqYxzUsoW

# Start an NNI example
! nnictl create --config nni_repo/nni/examples/trials/mnist-pytorch/config.yml --port 5000 &
# Start ngrok
get_ipython().system_raw('./ngrok http 5000 &')

# Get ngrok tunnel
! curl -s http://localhost:4040/api/tunnels

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
[2022-10-25 22:11:05] Creating experiment, Experiment ID: 04cba1pe
[2022-10-25 22:11:05] Starting web server...
[2022-10-25 22:11:06] Setting up...
[2022-10-25 22:11:07] Web portal URLs: http://127.0.0.1:5000 http://172.28.0.2:5000
[2022-10-25 22:11:07] To stop experiment run "nnictl stop 04cba1pe" or "nnictl stop --all"
[2022-10-25 22:11:07] Reference: https://nni.readthedocs.io/en/stable/reference/nnictl.html
{"tunnels":[],"uri":"/api/tunnels"}
